<a href="https://colab.research.google.com/github/BootCamp-BMA/colabs/blob/main/extract_DziriBert_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset


In [ ]:

# Load tokenizer and model
model_name = "alger-ia/dziribert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at alger-ia/dziribert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import pandas as pd
import re
from transformers import BertTokenizer

# Load dataset
train_df = pd.read_csv("FASSILA/Data/Fake News/train_set.csv")
test_df = pd.read_csv("FASSILA/Data/Fake News/test_set.csv")
val_df = pd.read_csv("FASSILA/Data/Fake News/validation_set.csv")

# Arabic text normalization
def clean_text(text):
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = re.sub(r"\d+", "", text)  # Remove numbers
    return text.strip()

# Apply cleaning
train_df["text"] = train_df["text"].apply(clean_text)
test_df["text"] = test_df["text"].apply(clean_text)
val_df["text"] = val_df["text"].apply(clean_text)

# Tokenize text using DziriBERT
tokenizer = BertTokenizer.from_pretrained("alger-ia/dziribert")
train_tokens = tokenizer(list(train_df["text"]), padding=True, truncation=True, return_tensors="pt")
test_tokens = tokenizer(list(test_df["text"]), padding=True, truncation=True, return_tensors="pt")
val_tokens = tokenizer(list(val_df["text"]), padding=True, truncation=True, return_tensors="pt")


In [ ]:
from transformers import BertModel
import torch

# Load DziriBERT model
model_name = "alger-ia/dziribert"
model = BertModel.from_pretrained(model_name)
model.eval()

# Function to extract embeddings
def extract_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # CLS token embedding

# Extract embeddings for train, test, validation
train_embeddings = extract_embeddings(train_df["text"].tolist())
test_embeddings = extract_embeddings(test_df["text"].tolist())
val_embeddings = extract_embeddings(val_df["text"].tolist())

# Save embeddings
pd.DataFrame(train_embeddings).to_csv("FASSILA/Features/Fake News/DziriBERT/train_embeddings.csv", index=False)
pd.DataFrame(test_embeddings).to_csv("FASSILA/Features/Fake News/DziriBERT/test_embeddings.csv", index=False)
pd.DataFrame(val_embeddings).to_csv("FASSILA/Features/Fake News/DziriBERT/val_embeddings.csv", index=False)


Some weights of BertModel were not initialized from the model checkpoint at alger-ia/dziribert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
